# HealthVest AI - Lab Report Analyzer

**MedGemma Impact Challenge Submission**

An AI-powered lab report analyzer that helps Indian patients understand their blood test results in plain English.

## Problem
- Patients struggle to understand medical jargon in lab reports
- Reference ranges are confusing without context
- No easy way to track health trends over time

## Solution
Upload blood test report → Get plain English explanations for each value

In [ ]:
# Install dependencies
!pip install -q transformers accelerate pillow pdf2image

In [ ]:
import torch
import json
from PIL import Image
from transformers import AutoProcessor, AutoModelForImageTextToText
from google.colab import userdata
import os

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Load MedGemma Model

Using MedGemma 1.5 4B - Google's open-source medical AI model.

In [ ]:
# Model configuration - MedGemma 1.5 (latest, Jan 2026)
MODEL_ID = "google/medgemma-1.5-4b-it"

# Get HF token from Kaggle secrets
# Add your token in Kaggle: Settings > Secrets > Add "HF_TOKEN"
try:
    HF_TOKEN = userdata.get('HF_TOKEN')
except:
    HF_TOKEN = os.environ.get('HF_TOKEN', None)

if not HF_TOKEN:
    print("⚠️ HF_TOKEN not found. Add it in Kaggle Secrets.")
else:
    print("✓ HF_TOKEN found")

In [ ]:
# Load processor and model
print("Loading MedGemma processor...")
processor = AutoProcessor.from_pretrained(
    MODEL_ID,
    token=HF_TOKEN,
    trust_remote_code=True
)

print("Loading MedGemma model (this takes 2-3 minutes)...")
model = AutoModelForImageTextToText.from_pretrained(
    MODEL_ID,
    token=HF_TOKEN,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

print("✓ MedGemma loaded successfully!")

## Extraction Prompt

Carefully crafted prompt for extracting lab values from Indian lab report formats.

In [ ]:
EXTRACTION_PROMPT = """You are a medical lab report analyzer. Extract all test values from this lab report image.

For each test, provide:
- test_name: Name of the test (e.g., "Hemoglobin", "Fasting Blood Sugar", "TSH")
- value: Numeric value as shown
- unit: Unit of measurement (e.g., "g/dL", "mg/dL", "mIU/L")
- reference_range: Normal range as shown on report
- status: "normal", "high", or "low" based on reference range

Return ONLY a JSON array. Example:
[
  {"test_name": "Hemoglobin", "value": 14.2, "unit": "g/dL", "reference_range": "13.0-17.0", "status": "normal"}
]

Extract ALL tests visible. Use exact values. Handle Indian lab formats (Thyrocare, SRL, Dr. Lal PathLabs).
"""

EXPLANATION_PROMPT = """You are a friendly medical educator. Explain this lab value simply:

Test: {test_name}
Value: {value} {unit}
Normal Range: {reference_range}
Status: {status}

In under 80 words, explain:
1. What this test measures
2. What your result means
3. One actionable tip (if needed)

Use simple language. Never diagnose - suggest discussing with doctor if abnormal.
"""

## Core Functions

In [ ]:
def extract_lab_values(image: Image.Image) -> list:
    """Extract lab values from a lab report image using MedGemma."""
    
    # Resize if needed
    max_size = 1024
    if max(image.size) > max_size:
        ratio = max_size / max(image.size)
        new_size = (int(image.size[0] * ratio), int(image.size[1] * ratio))
        image = image.resize(new_size, Image.Resampling.LANCZOS)
    
    # Prepare inputs
    inputs = processor(
        images=image,
        text=EXTRACTION_PROMPT,
        return_tensors="pt"
    ).to(model.device)
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=2048,
            do_sample=False
        )
    
    # Decode
    response = processor.decode(outputs[0], skip_special_tokens=True)
    
    # Parse JSON
    try:
        start = response.find('[')
        end = response.rfind(']') + 1
        if start != -1 and end > start:
            return json.loads(response[start:end])
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        print(f"Raw response: {response}")
    
    return []


def explain_lab_value(test_name: str, value: float, unit: str, 
                      reference_range: str, status: str) -> str:
    """Generate plain English explanation for a lab value."""
    
    prompt = EXPLANATION_PROMPT.format(
        test_name=test_name,
        value=value,
        unit=unit,
        reference_range=reference_range,
        status=status
    )
    
    inputs = processor(
        text=prompt,
        return_tensors="pt"
    ).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=True,
            temperature=0.7
        )
    
    response = processor.decode(outputs[0], skip_special_tokens=True)
    
    # Remove prompt from response
    if prompt in response:
        response = response.replace(prompt, "").strip()
    
    return response


def analyze_report(image: Image.Image) -> dict:
    """Full analysis: extract values + generate explanations."""
    
    print("Extracting lab values...")
    lab_values = extract_lab_values(image)
    print(f"Found {len(lab_values)} tests")
    
    results = []
    for i, val in enumerate(lab_values):
        print(f"Explaining {i+1}/{len(lab_values)}: {val.get('test_name', 'Unknown')}...")
        
        explanation = explain_lab_value(
            val.get('test_name', ''),
            val.get('value', 0),
            val.get('unit', ''),
            val.get('reference_range', 'N/A'),
            val.get('status', 'normal')
        )
        
        results.append({
            **val,
            'explanation': explanation
        })
    
    return {
        'total_tests': len(results),
        'normal': sum(1 for r in results if r.get('status') == 'normal'),
        'abnormal': sum(1 for r in results if r.get('status') in ['high', 'low']),
        'results': results
    }

## Test with Sample Lab Report

Upload a lab report image to test.

In [ ]:
# Upload sample lab report
from google.colab import files
import io

print("Upload a lab report image (PNG, JPG) or PDF:")
uploaded = files.upload()

for filename, content in uploaded.items():
    print(f"\nProcessing: {filename}")
    
    if filename.lower().endswith('.pdf'):
        from pdf2image import convert_from_bytes
        images = convert_from_bytes(content, first_page=1, last_page=1)
        image = images[0]
    else:
        image = Image.open(io.BytesIO(content)).convert('RGB')
    
    print(f"Image size: {image.size}")
    display(image.resize((400, int(400 * image.size[1] / image.size[0]))))

In [ ]:
# Run analysis
results = analyze_report(image)

print("\n" + "="*60)
print("ANALYSIS COMPLETE")
print("="*60)
print(f"Total tests: {results['total_tests']}")
print(f"Normal: {results['normal']}")
print(f"Abnormal: {results['abnormal']}")

In [ ]:
# Display results
from IPython.display import HTML, display

def display_results(results):
    html = "<div style='font-family: Arial, sans-serif;'>"
    
    for r in results['results']:
        status = r.get('status', 'normal')
        color = '#28a745' if status == 'normal' else '#dc3545' if status == 'high' else '#ffc107'
        badge = '✓ Normal' if status == 'normal' else '↑ High' if status == 'high' else '↓ Low'
        
        html += f"""
        <div style='border: 1px solid #ddd; border-left: 4px solid {color}; 
                    padding: 15px; margin: 10px 0; border-radius: 4px;'>
            <div style='display: flex; justify-content: space-between; align-items: center;'>
                <h3 style='margin: 0; color: #333;'>{r.get('test_name', 'Unknown')}</h3>
                <span style='background: {color}; color: white; padding: 4px 12px; 
                             border-radius: 20px; font-size: 12px;'>{badge}</span>
            </div>
            <p style='font-size: 24px; margin: 10px 0; color: #333;'>
                <strong>{r.get('value', 'N/A')}</strong> 
                <span style='font-size: 14px; color: #666;'>{r.get('unit', '')}</span>
            </p>
            <p style='color: #666; font-size: 13px; margin: 5px 0;'>
                Reference: {r.get('reference_range', 'N/A')}
            </p>
            <hr style='border: none; border-top: 1px solid #eee; margin: 10px 0;'>
            <p style='color: #444; line-height: 1.5;'>{r.get('explanation', 'No explanation available.')}</p>
        </div>
        """
    
    html += "</div>"
    display(HTML(html))

display_results(results)

## Summary

### What MedGemma Does Well
- Extracts structured data from lab report images
- Identifies test names, values, units, and reference ranges
- Classifies values as normal/high/low
- Generates patient-friendly explanations

### Impact
- Helps patients understand their health data
- Reduces anxiety from confusing medical jargon
- Empowers informed discussions with doctors

### Next Steps
- Add trend tracking (compare with previous reports)
- Support more lab report formats
- Build mobile-friendly web app

In [ ]:
# Save results to JSON
with open('analysis_results.json', 'w') as f:
    json.dump(results, f, indent=2)
print("Results saved to analysis_results.json")